In [1]:
import random

import torchvision.models as models
import torch.nn as nn
import pandas as pd
import torch
from torch.utils.data import Dataset
import cv2
import numpy as np
from torch.utils.data import DataLoader
import torch
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn
import torchvision.models as models

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-02-27 09:50:14.412682: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-27 09:50:14.460391: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following ins

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
device

device(type='cuda')

In [4]:
x = torch.Tensor([1, 2, 3]).to(device)
print(x.device)

cuda:0


In [5]:
#TODO Возможно нужно подбирать гиперпаораметры 
IMG_SIZE = 255
BATCH_SIZE = 2
EPOCHS = 100
SEQ_LENGTH = 100

In [6]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [7]:
class CustomDataset(Dataset):
    def __init__(self, root_path="", df_path="train.csv", img_size=224, SEQ_LENGTH=100, transform=None):
        self.SEQ_LENGTH = SEQ_LENGTH
        self.root_path = root_path
        self.img_size = img_size
        df = pd.read_csv(df_path)
        self.video_paths = df['path'].tolist()
        self.labels = df['label'].tolist()
        self.transform = transform
        unique_labels = sorted(set(self.labels))
        self.label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
        self.idx_to_label = {idx: label for label, idx in self.label_to_idx.items()}

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        cap = cv2.VideoCapture(self.video_paths[idx])
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        frames = []
        if total_frames >= self.SEQ_LENGTH:
            frame_indices = np.linspace(0, total_frames - 1, self.SEQ_LENGTH, dtype=int)
        else:
            frame_indices = np.tile(np.arange(total_frames), self.SEQ_LENGTH // total_frames + 1)[:self.SEQ_LENGTH]

        for i in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret:
                break
            frame_tensor = self.transform(frame).unsqueeze(0).to(device)
            frames.append(frame_tensor)

        while len(frames) < self.SEQ_LENGTH:
            frames.append(torch.zeros_like(frames[0]))

        frames_tensor = torch.cat(frames, dim=0)

        return frames_tensor, self.label_to_idx[self.labels[idx]]

In [8]:
train_dataset = CustomDataset(df_path="train.csv", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_dataset = CustomDataset(df_path="test.csv", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [9]:
train_dataset[0][0].shape

torch.Size([100, 3, 224, 224])

In [10]:
class CNN_RNN(nn.Module):
    def __init__(self, num_classes, rnn_hidden_size=256, rnn_num_layers=2):
        super(CNN_RNN, self).__init__()
        
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(2048, num_classes)
        ).to(device)

        # LSTM
        self.rnn = nn.LSTM(input_size=2, hidden_size=rnn_hidden_size, num_layers=rnn_num_layers, batch_first=True)
        # полносвязанный слой для классификации
        self.fc = nn.Linear(rnn_hidden_size, num_classes)

    def forward(self, x):
        # Прогоняем resNetку 
        batch_size, seq_length, _, h, w = x.size()
        x = x.view(batch_size * seq_length, 3, h, w)
        features = self.resnet(x)
        features = features.view(batch_size, seq_length, -1)
        # Прогоняем RNN
        rnn_out, _ = self.rnn(features)
        # Получаем последний выход RNN и применяем полносвязный слой для классификации
        output = self.fc(rnn_out[:, -1, :])
        return output

In [11]:
# TODO Подбор архитектуры, функции ошибки и оптимайзера........
model = CNN_RNN(2, rnn_hidden_size=8)

best_model_path = 'REsnet_for_images_results/best_model_images.pt'
model.resnet.load_state_dict(torch.load(best_model_path))

model = nn.DataParallel(model, device_ids = [ 0, 1, 2, 3]).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.9), eps=1e-08, weight_decay=0.0001)
writer = SummaryWriter()


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import logging
open('training2.log', 'w').close()
logging.basicConfig(filename='training2.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

best_accuracy = 0.0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if i % 10 == 9:  
            print('[%d, %5d] loss: %.7f accuracy: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100, correct / total))
            logging.info('[%d, %5d] loss: %.7f accuracy: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100, correct / total))
            # TODO добавить данные в борд 
            writer.add_scalar('Training Loss', running_loss / 100, epoch * len(train_loader) + i)
            writer.add_scalar('Training Accuracy', correct / total, epoch * len(train_loader) + i)
            running_loss = 0.0
            correct = 0
            total = 0
            
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        test_accuracy = correct / total
        logging.info('Accuracy of the network on the test images: %.3f' % (test_accuracy))
        print('Accuracy of the network on the test images: %.3f' % (test_accuracy))
        writer.add_scalar('Test Accuracy', test_accuracy, epoch)
        
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            torch.save(model.state_dict(), 'best_model.pth')
            logging.info('Model saved with accuracy: %.3f' % (best_accuracy))
            print('Model saved with accuracy: %.3f' % (best_accuracy))

print('Finished Training')

writer.close()


[1,    10] loss: 0.0685280 accuracy: 0.600
[1,    20] loss: 0.0743821 accuracy: 0.500
[1,    30] loss: 0.0676843 accuracy: 0.600
[1,    40] loss: 0.0824905 accuracy: 0.300
[1,    50] loss: 0.0632768 accuracy: 0.700
[1,    60] loss: 0.0714992 accuracy: 0.500
[1,    70] loss: 0.0690323 accuracy: 0.550
[1,    80] loss: 0.0730477 accuracy: 0.450
[1,    90] loss: 0.0721888 accuracy: 0.450
[1,   100] loss: 0.0673150 accuracy: 0.600
[1,   110] loss: 0.0656864 accuracy: 0.650
[1,   120] loss: 0.0671732 accuracy: 0.600
[1,   130] loss: 0.0709078 accuracy: 0.500
[1,   140] loss: 0.0717199 accuracy: 0.450
[1,   150] loss: 0.0668111 accuracy: 0.650
[1,   160] loss: 0.0670759 accuracy: 0.600
[1,   170] loss: 0.0690820 accuracy: 0.550
[1,   180] loss: 0.0658860 accuracy: 0.650
[1,   190] loss: 0.0741268 accuracy: 0.400
[1,   200] loss: 0.0717243 accuracy: 0.450
[1,   210] loss: 0.0703489 accuracy: 0.500
[1,   220] loss: 0.0702632 accuracy: 0.500
[1,   230] loss: 0.0727344 accuracy: 0.350
[1,   240] 